In [7]:
import pandas as pd
import numpy as np


In [8]:
df_app = pd.read_csv("./loan_defaulter/application_data.csv")
df_data = pd.read_csv("./loan_defaulter/previous_application.csv")

- Cek 1 user berkali2 pinjam atau tidak.
- Handling missing value dengan MAR (cek keterkaitan)
- Langkah penanganan loan terprediksi default
- Cari referensi penanganan nasabah pinjaman default